In [ ]:
# 2025.3.22 tue. updated by Kensuke Ohtake
using Random
using Distributions
using Plots
using Dates
using Format
using DataFrames
using CSV

function advection_diffusion_model(pls)

    SD = Dates.millisecond(now())
    Random.seed!(SD) # Set seed
    
    # control parameters
    mu = pls[1]# importance of manufacturing goods
    sig = pls[2]# elasticity of substitution of manufactured goods
    tau = pls[3]# manufacturing transportation cost parameter
    Lm = pls[4]# total manufacturing population
    
    function dist(z)
        return minimum([abs(z), 2.0*π - abs(z)])
    end
    
    I = 256 # Number of space fragments
    dx = 2.0 * π / convert(Float64, I) # size of space fragments
    dt = 0.01 # size of time step
    F = 1.0 # fixed cost
    eps = 1e-11 # error
    diff = 0.005 # diffusion coeficient for diffusion term
    conv = 0.5 # convection coeficient for convection term
    diffdtdx2 = diff * dt / (dx^2.0)
    convdtdx = conv * dt / dx
    
    x = zeros(Float64, I, 1)
    for i in 1:I
        x[i] = -π + (i-1) * dx
    end
    
    # Transport matrix
    Ex = zeros((I, I))
    for i in 1:I
        for j in 1:I
            Ex[i,j] = exp(- tau * (sig - 1.0) * dist(x[i]-x[j])) # for manufacturing
        end
    end
    
    # initial values of manufacturing workers and firms
    turb_lm = rand(Uniform(-0.0005, 0.0005), (I, 1))
    turb_lm = turb_lm .- (sum(turb_lm)*dx)/(2.0*π) # total 0
    
    lm = (Lm * ones((I, 1)) / (2.0*π)) + turb_lm # manufacturing workers
    ph = 10.0 * ones((I, 1)) / (2.0*π) # agricultural workers
    
    if sum(sign.(lm)) != I
        error("Non-negativity_is_not_satisfied_!!")
    elseif Lm - 1e-10 > sum(lm)*dx
        error("Conservation_error_!!")
    elseif sum(lm)*dx > Lm + 1e-10
        error("Conservation_error_!!")
    else
        ini_lm = copy(lm)
        println("Initial_total_manufacturing_workers_is_", sum(ini_lm)*dx)
    end
    
    # RealWage function
    function RealWageFunc(agr_workers, manu_workers)
        
        # inputs must be numpy array of which shape is (I, 1)
        
        # Get instantaneous equilibrium
        D = Ex * manu_workers
        P = (D*dx / F) .^ (1.0 / (1.0 - sig))# price index
        w = (mu / sig) * (Ex * ((agr_workers + manu_workers) ./ D))# manufacturing nominal wage
        if any(w .<= mu) == true
            error("Demand_for_agricultural_goods_error_!!")
        end
    
        rw = w - mu * log.(P)# real wage
    
        return rw
    end
    
    # time evolution  
    t = 0

    dlm = zeros((I, 1))
    
    while true
        lm_old = copy(lm)
        
        omega = RealWageFunc(ph, lm)
        vm = omega - circshift(omega, 1)# omega_j - omega_{j-1}
        vp = circshift(omega, -1) - omega# omega_{j+1} - omega_j
        fplus = ((vp + abs.(vp)) .* lm + (vp - abs.(vp)) .* circshift(lm, -1)) / (2.0 * dx) # numerical flux at j+1/2
        fminus = ((vm + abs.(vm)) .* circshift(lm, 1) + (vm - abs.(vm)) .* lm) / (2.0 * dx) # numerical flux at j-1/2
        dlm = diffdtdx2 * (circshift(lm, -1) - 2.0*lm + circshift(lm, 1)) - convdtdx * (fplus - fminus)
            
        lm = lm_old + dlm
 
        if sum(sign.(lm)) != I
            error("Non-negativity_is_not_satisfied_!!")   
        elseif Lm - 1e-10 > sum(lm)*dx
            error("Conservation_error_!!")
        elseif sum(lm)*dx > Lm + 1e-10
            error("Conservation_error_!!")
        end
        
        if maximum(abs.(dlm)) < eps
            break
        end
    
        t += 1
    
    end
    
    # obtain stationary solutions
    Intg_sum = Ex * lm
    Price_index = (Intg_sum*dx / F) .^ (1.0 / (1.0 - sig))# price index
    Nominal_wage = (mu / sig) * Ex * ((lm + ph) ./ Intg_sum)# Nominal wage
    if any(Nominal_wage .<= mu) == true
        error("Demand_for_agricultural_goods_error_!!")
    end
    Real_wage = Nominal_wage - mu * log.(Price_index)
    println("Final_total_population_is_", sum(lm)*dx)
    
    # Plot
    DateTime = Dates.format(now(), "yyyymmddHHMMSS")# get datetime
    plt_x = append!(reshape(x, I), π)
    plt_ini_lm = append!(reshape(ini_lm, I), ini_lm[1])
    plt_lm = append!(reshape(lm, I), lm[1])
    plt_Nominal_wage = append!(reshape(Nominal_wage, I), Nominal_wage[1])
    plt_Price_index = append!(reshape(Price_index, I), Price_index[1])
    plt_Real_wage = append!(reshape(Real_wage, I), Real_wage[1])
    
    filename_lm = format("{}-pop_tau{}sigma{}mu{}Lambda{}I{}t{}sd{}", DateTime, tau, sig, mu, Lm, I, t, SD)
    newfilename_lm = replace(filename_lm, "."=>"p")
    filename_rw = format("{}-rw_tau{}sigma{}mu{}Lambda{}I{}t{}sd{}", DateTime, tau, sig, mu, Lm, I, t, SD)
    newfilename_rw = replace(filename_rw, "."=>"p")
    
    plot(plt_x, plt_lm, title="manu_workers", label="", dpi=300)
    savefig(newfilename_lm * ".png")
    plot(plt_x, plt_Real_wage, title="realwage", label="", dpi=300)
    savefig(newfilename_rw * ".png")
    
    # Output and save data as csv
    df = DataFrame(coordinates = plt_x,
        initial_lambda = plt_ini_lm,
        final_lambda = plt_lm,
        nominal_wage = plt_Nominal_wage,
        price_index = plt_Price_index,
        real_wage = plt_Real_wage) # Save data as dataframe
    #df_ind = df.set_index('coordinates') # Make coodinates-column index
    filename = format("{}_tau{}sigma{}mu{}Lambda{}I{}sd{}", DateTime, tau, sig, mu, Lm, I, SD)
    newfilename = replace(filename, "."=>"p")
    CSV.write(newfilename * ".csv", df)# Write into csv

end


In [ ]:
# Loop
for ta in [0.05, 0.15, 0.25, 0.35, 0.37, 0.42]
    params = [0.6, 5.0, ta, 1.0]# set parameters [mu, sigma, tau, Lambda]
    for itr in 1:5
        advection_diffusion_model(params)
    end
end
